In [3]:
import numpy as np
import os


In [ ]:
def find_voc(scan_filename):
    arr = np.loadtxt(scan_filename, delimiter=",", dtype=str)
    header_row = np.where(arr == "Time")[0][0]

    meta_data = {}
    for data in arr[:header_row, :2]:
        meta_data[data[0]] = data[1]

    arr = arr[header_row + 1 :, :]

    data = arr[:, 2:-1]

    pixel_V = data[:, ::2][:, ::-1].astype(float)
    pixel_mA = data[:, 1::2][:, ::-1].astype(float)
    print(len(pixel_V))
    voc = []
    for pixel_idx in range(8):
        voc_idx = min(
                range(len(pixel_mA[:, pixel_idx])),
                key=lambda x: abs(pixel_mA[:, pixel_idx][x]),
            )
        voc.append(float(pixel_V[voc_idx, pixel_idx]))

    return voc

print(
    find_voc(
        r"C:\Users\achen\Dropbox\code\Stability-Setup\data\april_1_test\Apr-01-2025 21_21_13\Apr-01-2025_21-43-05__ID1__light__scan.csv"
    )
)

82
[0.9249, 0.9227, 0.9026, 0.9202, 0.971, 0.9474, 0.9201, 0.9754]


In [4]:
arr = np.loadtxt(r"C:\Users\achen\Dropbox\code\Stability-Setup\data\april_1_test\Apr-01-2025 21_21_13\Apr-01-2025_21-22-03__ID1__mppt.csv", delimiter=",", dtype=str)
header_row = np.where(arr == "Time")[0][0]

meta_data = {}
for data in arr[:header_row, :2]:
	meta_data[data[0]] = data[1]

headers = arr[header_row, :]
arr = arr[header_row + 1 :, :]

header_dict = {value: index for index, value in enumerate(headers)}

pixel_V = arr[:, ::2][:, ::-1].astype(float)
pixel_mA = arr[:, 1::2][:, ::-1].astype(float)
time = np.array(arr[:, header_dict["Time"]]).astype("float")

data = pixel_V*pixel_mA

print(data)

[[6.83373278e+08 6.25892136e+00 6.97118298e+00 ... 1.71600000e-01
  1.82000000e-01 9.08400000e-02]
 [7.09706295e+08 6.70273474e+00 7.54437663e+00 ... 1.74000000e-01
  1.84600000e-01 1.83120000e-01]
 [7.37425260e+08 7.22165304e+00 8.00309392e+00 ... 1.75200000e-01
  1.85900000e-01 2.98480000e-01]
 ...
 [3.75448338e+09 2.12301475e+02 2.44667245e+02 ... 9.57600000e-01
  1.02750000e+00 1.26882680e+02]
 [3.74505893e+09 2.12740349e+02 2.45284760e+02 ... 9.50400000e-01
  1.02000000e+00 1.27430090e+02]
 [3.74853640e+09 2.13276447e+02 2.44709915e+02 ... 9.44300000e-01
  1.02000000e+00 1.27976790e+02]]


In [ ]:
arrs = [
    r"C:\Users\achen\Dropbox\code\Stability-Setup\data\Feb-21-2025 21_27_25\Feb-21-2025 21_27_39lightID1scan.csv",
    r"C:\Users\achen\Dropbox\code\Stability-Setup\data\Feb-21-2025 21_29_06\Feb-21-2025 21_29_07lightID1scan.csv",
    r"C:\Users\achen\Dropbox\code\Stability-Setup\data\Feb-21-2025 21_30_48\Feb-21-2025 21_30_51lightID1scan.csv",
    r"C:\Users\achen\Dropbox\code\Stability-Setup\data\Feb-21-2025 21_36_55\Feb-21-2025 21_36_57lightID1scan.csv",
    r"C:\Users\achen\Dropbox\code\Stability-Setup\data\Feb-21-2025 21_39_57\Feb-21-2025 21_39_59lightID1scan.csv",
    r"C:\Users\achen\Dropbox\code\Stability-Setup\data\Feb-21-2025 21_41_06\Feb-21-2025 21_41_07lightID1scan.csv",

]

# 0.0435
avg = []
for i in arrs:
arr = np.loadtxt(i, delimiter=",", dtype=str)
time = arr[9:,0].astype(float)
diffs = np.diff(time)
positive_diffs = diffs[diffs >= 0]
avg.append(np.mean(positive_diffs)/5)

print(np.mean(avg))


# '''
# 1.2v/0.03v = 40 measurements
# v_Range / step size = x
# 1000* 1.2v / 0.05v/s = 2400ms
# v_Range / rate  = y
# 24s / 40 = 0.6 seconds per measurement
# x / y = z
# 600 ms per measurement
# 0.6 - 0.00868*measurements
# delay_time_ms = z - 0.00868 - measurement_count * measurement_delay
# '''

0.01758375


In [26]:
folder_path = (
    r"C:\Users\achen\Dropbox\code\Stability-Setup\data\Feb-25-2025 12_46_22__file test"
)
csv_files = sorted(
    [
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if f.lower().endswith(".csv")
    ]
)

def getPlotGroups(csv_files):
    file_groups_dict = {}
    for file in csv_files:
        head, tail = os.path.split(file)
        params = tail.split("__")
        filetype = params[-1].split(".")[0]  # get scan or mppt from scan.csv
        if "ID" not in params[1]:
            test_name = params[1]
        else:
            test_name = ""

        name_parts = [val for val in [test_name, params[0], filetype] if val]

        plot_name = " ".join(name_parts)
        if plot_name in file_groups_dict:
            file_groups_dict[plot_name].append(file)
        else:
            file_groups_dict[plot_name] = [file]

    return file_groups_dict


for i in getPlotGroups(csv_files):
    print(i, getPlotGroups(csv_files)[i])

Feb-25-2025_12-46-26 scan ['C:\\Users\\achen\\Dropbox\\code\\Stability-Setup\\data\\Feb-25-2025 12_46_22__file test\\Feb-25-2025_12-46-26__ID1__light__scan.csv', 'C:\\Users\\achen\\Dropbox\\code\\Stability-Setup\\data\\Feb-25-2025 12_46_22__file test\\Feb-25-2025_12-46-26__ID2__light__scan.csv']
file test Feb-25-2025_12-46-26 scan ['C:\\Users\\achen\\Dropbox\\code\\Stability-Setup\\data\\Feb-25-2025 12_46_22__file test\\Feb-25-2025_12-46-26__file test__ID1__light__scan.csv', 'C:\\Users\\achen\\Dropbox\\code\\Stability-Setup\\data\\Feb-25-2025 12_46_22__file test\\Feb-25-2025_12-46-26__file test__ID2__light__scan.csv']
file test Feb-25-2025_12-47-32 mppt ['C:\\Users\\achen\\Dropbox\\code\\Stability-Setup\\data\\Feb-25-2025 12_46_22__file test\\Feb-25-2025_12-47-32__file test__ID1__mppt.csv', 'C:\\Users\\achen\\Dropbox\\code\\Stability-Setup\\data\\Feb-25-2025 12_46_22__file test\\Feb-25-2025_12-47-32__file test__ID2__mppt.csv']


In [ ]:
params = ['0.3750','0.00','0.0239']
command = "scan," + ",".join(params)

print(command)

scan,0.3750, 0.00, 0.0239
